In [59]:
import carla
import numpy as np
import cv2


In [60]:
import random
import time
from PIL import Image
import pandas as pd


In [61]:
client=carla.Client("localhost",2000)
world=client.get_world()
steer_angle_info=[]
throttle_in_info=[]
brake_in_info=[]
save_img_info=[]
steer_angle_info1=[]
throttle_in_info1=[]
brake_in_info1=[]
save_img_info1=[]

In [62]:
allspawn=[]

In [63]:
blueprint_library = world.get_blueprint_library()
spawnpoints=world.get_map().get_spawn_points()

In [64]:
ar=blueprint_library.find("vehicle.mercedes.coupe_2020")
transforms1=random.choice(spawnpoints)
allspawn.append(transforms1)
car=world.try_spawn_actor(ar,transforms1)

In [65]:
ar=blueprint_library.find("vehicle.mercedes.coupe_2020")
while True:
    transforms2=random.choice(spawnpoints)
    if transforms2 not in allspawn:
        allspawn.append(transforms2)
        break
car1=world.try_spawn_actor(ar,allspawn[1])

In [66]:
spectator=world.get_spectator()
x=carla.Transform(car1.get_transform().transform(carla.Location(x=-0.4,z=1.8)),car1.get_transform().rotation)
spectator.set_transform(x)

In [67]:
cam1=blueprint_library.find("sensor.camera.rgb")
cam1.set_attribute("image_size_x","1920")
cam1.set_attribute("image_size_y","1080")
cam1.set_attribute("fov","110")
cam1.set_attribute("sensor_tick","0.5")
transforms=carla.Transform(carla.Location(z=1.8,x=-.2))
camera=world.spawn_actor(cam1,transforms,attach_to=car)


In [68]:
cam=blueprint_library.find("sensor.camera.rgb")
cam.set_attribute("image_size_x","1920")
cam.set_attribute("image_size_y","1080")
cam.set_attribute("fov","110")
cam.set_attribute("sensor_tick","0.5")
transforms=carla.Transform(carla.Location(z=1.8,x=-.2))
camera1=world.spawn_actor(cam,transforms,attach_to=car1)



In [69]:
car1.set_autopilot()



In [70]:
car.set_autopilot()

In [71]:
def show_me(image):
    r=np.array(image.raw_data)
    r1=r.reshape((1080,1920,4))
    r2=r1[:,:,:3]
    im=cv2.cvtColor(r2,cv2.COLOR_BGR2RGB)
    im=Image.fromarray(im)
    A=image.frame
    im.save('pokemon4/%06d.png'%A)
    save_img_info.append('pokemon4/%06d.png'%A)
    steer_angle_info.append(car.get_control().steer)
    throttle_in_info.append(car.get_control().throttle)
    brake_in_info.append(car.get_control().brake)
    #camera_data["zs"]=r2
    

In [72]:
def show_me1(image):
    r=np.array(image.raw_data)
    r1=r.reshape((1080,1920,4))
    r2=r1[:,:,:3]
    im=cv2.cvtColor(r2,cv2.COLOR_BGR2RGB)
    im=Image.fromarray(im)
    A=image.frame
    im.save('pokemon5/%06d.png'%A)
    save_img_info1.append('pokemon4/%06d.png'%A)
    steer_angle_info1.append(car1.get_control().steer)
    throttle_in_info1.append(car1.get_control().throttle)
    brake_in_info1.append(car1.get_control().brake)

    

In [73]:
a=time.time()

In [74]:

camera.listen(lambda image:show_me1(image))

  

In [75]:
camera1.listen(lambda image:show_me(image))




In [76]:
def updater(image,lis1,lis2,lis3):
    if len(image)>len(lis1):
        image.pop()
    elif len(lis1)>len(lis2):
        image.pop()
        lis1.pop()
    elif len(lis2)>len(lis3):
        image.pop()
        lis1.pop()
        lis2.pop()                    

In [77]:
time.sleep(10)
updater(save_img_info,throttle_in_info,steer_angle_info,brake_in_info)
df=pd.DataFrame({"cam_path":save_img_info,"Throttle":throttle_in_info,"Steering":steer_angle_info,"Braking":brake_in_info})
updater(save_img_info1,throttle_in_info1,steer_angle_info1,brake_in_info1)
df1=pd.DataFrame({"cam_path":save_img_info1,"Throttle":throttle_in_info1,"Steering":steer_angle_info1,"Braking":brake_in_info1})
df.to_csv(r"D:\\Carla Dataset formation\\data2.csv")
df1.to_csv(r"D:\\Carla Dataset formation\\data1.csv")
camera1.destroy()
car1.destroy()
car.destroy()
camera.destroy()

True